# Build an Interactice Finance Agent

This notebook leverages all the skills you've learned so far to build an interactive agent that can securely access our financial databases. First, you will configure a new flexible toolset for MCP Toolbox. Then you will deploy a new version of the Finance Agent that is available to access over public IP, secured by IAP to prevent unauthorized access. Finally, you will use the built-in ADK Agent UI to interact with the agent to answer ad-hoc questions about transactions, credit cards, and account transfers. 

> IMPORTANT: The notebooks in this repo build on top of one another. Be sure to run the preceding notebooks before you run this one. 

## Basic Setup

### Define Notebook Variables

Update the variables below to match your environment. You will be prompted for the AlloyDB password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# AlloyDB variables
alloydb_cluster = "my-alloydb-cluster"
alloydb_instance = "my-alloydb-instance"
alloydb_database = "finance"

# Spanner variables
spanner_instance = "my-spanner-instance"
spanner_database = "finance-graph"

# Cloud SQL Variables
cloud_sql_instance = "my-postgres-instance"
cloud_sql_database = "postgres"
cloud_sql_user = "postgres"
cloud_sql_password = input("Please enter the password for the Cloud SQL 'postgres' database user: ")

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet toolbox-core==0.2.1

### Define Helper Functions

#### get_auth_token()

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

    return id_token

#### OIDC Helpers

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2 import id_token
from google.cloud import iam_admin_v1
from google.cloud import resourcemanager_v3


def get_project_number(project_id: str) -> str:
    # Create a client for the Resource Manager API.
    client = resourcemanager_v3.ProjectsClient()

    # Construct the project name resource string.
    project_name = f"projects/{project_id}"

    # Create the request to get the project details.
    request = resourcemanager_v3.GetProjectRequest(name=project_name)

    # Make the API call to retrieve the project.
    project = client.get_project(request=request)

    # The project number is a string, not an integer.
    return project.name.split('/')[1]


def get_service_account_oauth2_client_id(project_id: str, service_account_email: str) -> str:
    # Reference: https://cloud.google.com/iap/docs/authentication-howto#python

    # Construct the full service account name resource string.
    name = f"projects/{project_id}/serviceAccounts/{service_account_email}"

    # Create the IAM Admin client.
    iam_admin_client = iam_admin_v1.IAMClient()

    # Prepare the request to get the service account.
    request = iam_admin_v1.GetServiceAccountRequest(name=name)

    # Make the API call.
    service_account = iam_admin_client.get_service_account(request=request)

    # Return the oauth2_client_id attribute.
    # This will be an empty string if it doesn't exist.
    return service_account.oauth2_client_id


def get_oidc_token(project_id, service_account_email):
    # Reference: https://cloud.google.com/iap/docs/authentication-howto#python
    oauth_client_id = get_service_account_oauth2_client_id(project_id, service_account_email)
    return id_token.fetch_id_token(Request(), oauth_client_id)


## Update the MCP Toolbox Toolset

The goal of our interactive agent is to be as flexible as possible, so we will use an alloydb_nl tool to flexibly and securely interact with our AlloyDB database. We will also build tools that will allow us to ask questions about account transfers that might be related to fraudulent transactions. 

### Update the `tools.yaml` File

Read through the definitions of each tool to understand the purpose of each query.

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/resources/sources/spanner/
#            https://googleapis.github.io/genai-toolbox/resources/tools/
#            https://googleapis.github.io/genai-toolbox/resources/tools/spanner-sql/
#            https://googleapis.github.io/genai-toolbox/resources/sources/alloydb-pg/
#            https://googleapis.github.io/genai-toolbox/resources/tools/postgres-sql/

import os
import json

nl_config = "finance_agent_config"

tools_config = {
  "sources": {
    "spanner-finance-graph-source": {
      "kind": "spanner",
      "project": f"{project_id}",
      "instance": f"{spanner_instance}",
      "database": f"{spanner_database}",
      "dialect": "googlesql"
    },
    "alloydb-finance-source": {
      "kind": "alloydb-postgres",
      "project": f"{project_id}",
      "region": f"{region}",
      "cluster": f"{alloydb_cluster}",
      "instance": f"{alloydb_instance}",
      "database": f"{alloydb_database}",
      "user": "postgres",
      "password": "${ALLOYDB_PASSWORD}",
      "ipType": "private"
    }
  },
  "tools": {
    "get_loan_repayments": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get information about loans paid by customers from accounts they own. The query finds the accounts owned by the customer, and then discover the loans repaid by their accounts",
      "statement": """
        GRAPH FinGraph
        MATCH
          (person:Person {id: @customer_id})-[own:Owns]->
          (account:Account)-[repay:Repays]->(loan:Loan)
        RETURN
          person.id AS customer_id,
          person.name AS customer_name,
          account.id AS account_id,
          repay.create_time AS repay_time,
          repay.amount AS loan_repay_amount,
          loan.id AS loan_id,
          loan.loan_amount AS loan_amount,
          SAFE_TO_JSON(person) AS person_json, 
          SAFE_TO_JSON(own) AS own_json, 
          SAFE_TO_JSON(account) AS account_json, 
          SAFE_TO_JSON(repay) AS repay_json, 
          SAFE_TO_JSON(loan) AS loan_json
        ORDER BY repay.create_time DESC;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_indirect_transfers": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to check if money was moved between multiple accounts on its way from a given source account to a given destination account. The query matches all account money transfers starting from a source account id within 3 to 6 hops, to reach a destination account with another id. The {3,6} syntax is used to represent a quantified 3 to 6 hop path traversal between src_accnt and dst_accnt.",
      "statement": """
        GRAPH FinGraph
        MATCH
          (src_accnt:Account {id:@source_account_id})-[transfers:Transfers]->{3,6}
          (dst_accnt:Account {id:@destination_account_id})
        RETURN
          ARRAY_LENGTH(transfers) AS num_hops,
          src_accnt.type as source_account_type,
          src_accnt.is_blocked AS source_is_blocked,
          src_accnt.create_time AS source_create_time,
          dst_accnt.type as destination_account_type,
          dst_accnt.is_blocked AS destination_is_blocked,
          dst_accnt.create_time AS destinatoin_create_time,
          TO_JSON(transfers) AS transfer_edges;
        """,
      "parameters": [
        {
          "name": "source_account_id",
          "type": "integer",
          "description": "Unique account id number for source account"
        },
        {
          "name": "destination_account_id",
          "type": "integer",
          "description": "Unique account id number for destination account"
        }
      ]
    },
    "get_account_inflows_outflows": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to understand the money flow of accounts owned by a specific customer id to look for suspicious large inflows or outflows as part of compliance analysis requirements.",
      "statement": """
        GRAPH FinGraph
        MATCH (person:Person {id: @customer_id})-[:Owns]->(accnt:Account)
        RETURN person, accnt

        NEXT

        MATCH (accnt)<-[inflow:Transfers]-(:Account)
        WHERE inflow.amount > 100
          AND inflow.create_time > TIMESTAMP("2020-1-1")
        RETURN person, accnt, SUM(inflow.amount) AS total_inflow_amounts
        GROUP BY person, accnt

        NEXT

        MATCH (accnt) -[outflow:Transfers]->(:Account)
        WHERE outflow.amount > 100
          AND outflow.create_time > TIMESTAMP("2020-1-1")
        RETURN person, accnt, total_inflow_amounts, SUM(outflow.amount) AS total_outflow_amounts
        GROUP BY person, accnt, total_inflow_amounts

        NEXT

        LET money_flow_ratio = total_inflow_amounts / total_outflow_amounts
        RETURN 
          person.name AS customer_name,
          person.id AS customer_id,
          accnt.id AS account_id, 
          accnt.type AS account_type,
          accnt.is_blocked,
          total_inflow_amounts, 
          total_outflow_amounts,
          money_flow_ratio
        ORDER BY money_flow_ratio DESC;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_destination_account_audit_events": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to find audit events of reached destination accounts that have received wire transfers from a suspicious account.",
      "statement": """
        SELECT
          audit.id as accnt_id,
          audit.audit_timestamp as audit_ts,
          audit.audit_details as details,
          accnt_transfers.*
        FROM
          AccountAudits audit,
          GRAPH_TABLE(
            FinGraph
            MATCH (suspicious_account:Account {id:@suspicious_account_id})-[:Transfers]->{1,2}(dest_accnt)
            RETURN DISTINCT dest_accnt.id AS reached_account_id, 
              dest_accnt.type AS reached_account_type, 
              dest_accnt.is_blocked AS reached_account_is_blocked, 
              dest_accnt.create_time AS reached_account_create_time,
              suspicious_account.type AS suspicious_account_type, 
              suspicious_account.is_blocked AS suspicious_account_is_blocked, 
              suspicious_account.create_time AS suspicious_account_create_time
          ) AS accnt_transfers
        WHERE accnt_transfers.reached_account_id = audit.id
        ORDER BY audit_ts DESC
        """,
      "parameters": [
        {
          "name": "suspicious_account_id",
          "type": "integer",
          "description": "Unique account id number for an account with suspicious activity"
        }
      ]
    },
    "get_finance_database_context": {
      "kind": "alloydb-ai-nl",
      "source": "alloydb-finance-source",
      "description": "Use this tool to look up information about financial transactions, credit cards, customers, mcc codes, and historical fraud labels.",
      "nlConfig": f"{nl_config}"
    }
  },
  "toolsets": {
    "interactive-tools": [
      "get_loan_repayments",
      "get_indirect_transfers",
      "get_account_inflows_outflows",
      "get_destination_account_audit_events",
      "get_finance_database_context"
    ]
  }
}

with open("tools.yaml", "w") as file:
    file.write(json.dumps(tools_config))


### Write Updated `tools.yaml` to Secret Manager

In [ ]:
# Create the secret
! gcloud secrets versions add tools --data-file=tools.yaml

In [ ]:
# Clean up the local file
import os
os.remove('tools.yaml')

### Update Toolbox with New `tools.yaml` File

In [ ]:
# Reference: https://cloud.google.com/sdk/gcloud/reference/run/services/update

! gcloud run services update toolbox --no-user-output-enabled \
    --update-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --region={region}
    

### Redeploy Toolbox with IAP

In [ ]:
# Define Toolbox Container Image
image = 'us-central1-docker.pkg.dev/database-toolbox/toolbox/toolbox:latest'

# Deploy to Cloud Run
! gcloud beta run deploy toolbox --no-user-output-enabled \
    --image={image} \
    --network={vpc} \
    --subnet={vpc} \
    --network-tags=toolbox \
    --service-account=toolbox-identity \
    --region={region} \
    --allow-unauthenticated \
    --set-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --args="--tools-file=/app/tools.yaml","--address=0.0.0.0","--port=8080","--telemetry-gcp" \
    --vpc-egress=all-traffic \
    --ingress=internal \
    --min=1 \
    --iap


### Test Tools

Since we now have a more complex set of tools that require various parameters, we'll define a list of tools and the parameters we should call for each one, then we'll iterate through each one using the `load_tool()` function to get and test specific tools. 

In [ ]:
import json
import toolbox_core
from toolbox_core import ToolboxClient, auth_methods

# Get endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Refresh auth_token
def get_auth_token(endpoint):
    # Cloud Run uses your service's hostname as the `audience` value
    # audience = 'https://my-cloud-run-service.run.app/'
    # For Cloud Run, `endpoint` is the URL (hostname + path) receiving the request
    # endpoint = 'https://my-cloud-run-service.run.app/my/awesome/url'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, f"{endpoint}")

    return id_token

auth_token = get_auth_token(toolbox_url)
auth_token_provider = auth_methods.aget_google_id_token # can also use sync method

# Define tools to test and arguments
tools_to_test = [
    {
        "tool": "get_loan_repayments",
        "params": {
            "customer_id": 263
        }
    },
    {
        "tool": "get_indirect_transfers",
        "params": {
            "source_account_id": 75,
            "destination_account_id": 199
        }
    },
    {
        "tool": "get_account_inflows_outflows",
        "params": {
            "customer_id": 330
        }
    },
    {
        "tool": "get_destination_account_audit_events",
        "params": {
            "suspicious_account_id": 11
        }
    },
    {
        "tool": "get_finance_database_context",
        "params": "How many users are in the database?"
    }
]

# Run tools 
async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    for t in tools_to_test:
        print(f"\n\nTesting tool: { t.get('tool') }, with Params: { t.get('params') }")
        tool = await client.load_tool(t.get('tool'))
        params = t.get('params')
        if isinstance(params, dict):
            result = await tool(**params)
        else:
            result = await tool(params)

        json_result = json.loads(result)
        print("\nTool result:")
        print(json.dumps(json_result, indent=2))
    


## Deploy the Interactive ADK Agent

This process is the same as the agent we configured in notebook 3, but this time we exposing the service to public IP traffic and protecting it with [Identity-Aware Proxy](https://cloud.google.com/iap/docs/enabling-cloud-run). We'll also update the agent to use the new `interactive-tools` toolset that we just deployed to MCP Toolbox.

References:
- https://github.com/alphinside/deploy-and-manage-adk-service/tree/main/weather_agent
- https://codelabs.developers.google.com/deploy-manage-observe-adk-cloud-run#1

### Use Cloud SQL for Session Persistence

The Agent Development Kit (ADK) uses a Session Service to maintain the state of conversations. By default, it uses an in-memory session service, but this is not suitable for production environments where the server may restart, and sessions would be lost. To ensure persistence, we can connect the ADK to a Cloud SQL database to act as the persistent store for the Session service.

### Create `__init__.py`

In [ ]:
# Initialize the agent directory
! mkdir -p deploy
! mkdir -p deploy/finance_agent

In [ ]:
%%writefile deploy/finance_agent/__init__.py
# __init__.py

from .agent import root_agent

__all__ = ["root_agent"]

### Create `agent.py`

In [ ]:
%%writefile deploy/finance_agent/agent.py
import os
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
from google.adk.agents import Agent
from toolbox_core import ToolboxSyncClient
from opentelemetry.instrumentation.aiohttp_client import (AioHttpClientInstrumentor)


def get_auth_token(endpoint: str) -> str:
    """Fetches an OIDC token for authenticating with a Cloud Run service."""
    auth_req = google.auth.transport.requests.Request()
    return google.oauth2.id_token.fetch_id_token(auth_req, endpoint)

# The URL for the toolbox service should be passed as an environment variable
# for better flexibility between different environments (dev, prod).
TOOLBOX_URL = os.getenv("TOOLBOX_URL")
if not TOOLBOX_URL:
    raise ValueError("TOOLBOX_URL environment variable not set.")

# Get an auth token to invoke the Toolbox on Cloud Run
auth_token = get_auth_token(TOOLBOX_URL)

# Initialize the Toolbox client once and reuse it
print(f"Connecting to toolbox server at {TOOLBOX_URL}")
toolbox_client = ToolboxSyncClient(
    TOOLBOX_URL,
    client_headers={"Authorization": f"Bearer {auth_token}"}
)

# Define the agent's instructions
prompt = """
You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
account details, loan payments, account transfers,suspicious activity reports, and other tasks 
related to financial data.
"""

# Define the root agent
root_agent = Agent(
    model='gemini-2.5-flash',
    name='interactive_finance_agent',
    description='A helpful AI assistant for financial tasks.',
    instruction=prompt,
    # Load the tools from your MCP Toolbox
    tools=toolbox_client.load_toolset("interactive-tools"),
)

# Enable the AioHttp library for Toolbox tracing
AioHttpClientInstrumentor().instrument()

### Create `server.py`

In [ ]:
%%writefile deploy/server.py
import os
from fastapi import FastAPI
from google.adk.cli.fast_api import get_fast_api_app
from google.cloud import logging as google_cloud_logging

# OTEL imports
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider, export
from opentelemetry.propagate import set_global_textmap
from opentelemetry.propagators.cloud_trace_propagator import CloudTraceFormatPropagator
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter

# Initialize the Google Cloud Logging client
logging_client = google_cloud_logging.Client()
logger = logging_client.logger(__name__)

# The directory where your agent.py and __init__.py are located
AGENT_DIR = os.path.dirname(os.path.abspath(__file__))

# Get the session service URI from environment variables.
# This is where you'll provide the connection string for Cloud SQL.
session_uri = os.getenv("SESSION_SERVICE_URI")

# Prepare arguments for the FastAPI app
app_args = {"agents_dir": AGENT_DIR, "web": True}

# Use the Cloud SQL session service if the URI is provided
if session_uri:
    app_args["session_service_uri"] = session_uri
    logger.log_text(f"Using database session service.", severity="INFO")
else:
    logger.log_text(
        "SESSION_SERVICE_URI not provided. Using in-memory session service. "
        "Sessions will be lost on server restart.",
        severity="WARNING",
    )
    
# Observability
provider = TracerProvider()
processor = export.BatchSpanProcessor(CloudTraceSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
print("Set trace provider")

# Create the FastAPI app using the ADK utility
app: FastAPI = get_fast_api_app(**app_args)

app.title = "finance-agent"
app.description = "API for interacting with the ADK Finance Agent"

# Main execution
if __name__ == "__main__":
    import uvicorn
    
    # Enable the AioHttp library for Toolbox tracing
    AioHttpClientInstrumentor().instrument()

    # Instrument the FastAPI app.
    # This automatically creates parent spans for requests and propagates the trace context.
    FastAPIInstrumentor.instrument_app(app)

    uvicorn.run(app, host="0.0.0.0", port=8080)
    
    

### Create `pyproject.toml`

In [ ]:
%%writefile deploy/pyproject.toml
[project]
name = "deploy-and-manage-adk-service"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "google-adk==1.5.0",
    "locust==2.37.10",
    "pg8000==1.31.2",
    "python-dotenv==1.1.0",
    "toolbox-core==0.2.1",
    "google-genai==1.23.0",
    "opentelemetry-instrumentation-fastapi==0.55b1",
    "opentelemetry-api>=1.20.0",
    "opentelemetry-sdk>=1.20.0",
    "opentelemetry-instrumentation-fastapi>=0.40.0",
    "opentelemetry-exporter-gcp-trace>=1.9.0",
    "opentelemetry-propagator-gcp==1.9.0",
    "opentelemetry-exporter-gcp-monitoring==1.9.0a0",
    "opentelemetry-exporter-otlp",
    "opentelemetry-instrumentation-requests==0.55b1",
    "opentelemetry-instrumentation-aiohttp-client"
]

[dependency-groups]
dev = [
    "pytest==8.4.0",
    "ruff==0.11.13",
]

### Create a Dockerfile

In [ ]:
%%writefile deploy/Dockerfile
FROM python:3.12-slim
RUN pip install --no-cache-dir uv==0.7.13
WORKDIR /app
COPY . .
RUN uv sync
EXPOSE 8080
CMD ["uv", "run", "uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8080"]


### Deploy the Agent to Cloud Run

Since this Cloud Run service interacts with databases, we want to be mindful of security best practices during deployment. Specifically:
- We provision the Cloud Run instnace in the `demo-vpc` private VPC so that it interacts with the MCP Toolbox and the Cloud SQL instance securely over the VPC.
- We set `--ingress=all` and `--no-allow-unauthenticated` this time so that we can access the interactive ADK UI. We will secure this connection with IAP.
- We use Secret Manager to securely store our MCP Toolbox tool definitions and our AlloyDB password, and we securely import those secrets into Cloud Run using `--set-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest"`
- We leverage private DNS resolution and Google Private Access (which we setup in notebook 3) for secure communication between the Agent and MCP Toolbox Cloud Run services.

In [ ]:
# Get the MCP Toolbox endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Get Cloud SQL connection name
cloud_sql_psc_ip = !gcloud compute forwarding-rules describe psc-forwarding-rule-{cloud_sql_instance} \
    --region={region} \
    --project={project_id} \
    --format="value(IPAddress)"
cloud_sql_psc_ip = cloud_sql_psc_ip[0]

# Construct the SESSION_SERVICE_URI
session_service_uri = f"postgresql+pg8000://postgres:{cloud_sql_password}@{cloud_sql_psc_ip}:5432/postgres"

print(f"SESSION_SERVICE_URI: {session_service_uri}")

!gcloud beta run deploy finance-agent --quiet --no-user-output-enabled \
    --source ./deploy/ \
    --network={vpc} \
    --subnet={vpc} \
    --ingress=all \
    --port 8080 \
    --project {project_id} \
    --region {region} \
    --allow-unauthenticated \
    --update-env-vars=TOOLBOX_URL={toolbox_url},GOOGLE_GENAI_USE_VERTEXAI=True,SESSION_SERVICE_URI={session_service_uri},GOOGLE_CLOUD_PROJECT={project_id},GOOGLE_CLOUD_LOCATION={region},GCS_BUCKET_NAME={gcs_bucket_name} \
    --min=1 \
    --memory=1Gi \
    --iap

### Grant finance-agent Invoke Permissions on toolbox

In [ ]:
# Get account finance-agent service account
service_account = !gcloud run services describe finance-agent \
    --region={region} \
    --format="value(spec.template.spec.serviceAccountName)"
service_account = service_account[0]

In [ ]:
# Grant service account invoke permissions on toolbox
!gcloud run services add-iam-policy-binding toolbox --region={region} \
    --member=serviceAccount:{service_account} \
    --role="roles/run.invoker"

## Enable Identity-Aware Proxy (IAP)

We previously restricted non-private ingress traffic on our ADK Agent hosted on Cloud Run. This is a great security practice for autonomous agents that are triggered by an event and take automated actions on your behalf. For interactive agents, however, it is useful to provide a friendly UI for Agent interactions, and the ADK framework provides a built-in UI specifically for this purpose. To make this UI available without sacrificing our security posture, we will enable IAP to prevent unauthorized access to the Agent, while allowing authorized individuals to safely connect. 

Reference: 
- https://cloud.google.com/iap/docs/enabling-cloud-run

### Setup IAP

In [ ]:
# Create the IAP service agent by running the following command
!gcloud beta services identity create --service=iap.googleapis.com --project={project_id}

In [ ]:
# Get project number
project_number = ! gcloud projects describe {project_id} --format='value(projectNumber)'
project_number = project_number[0]

# Grant the invoker permission to the service account
!gcloud run services add-iam-policy-binding finance-agent \
    --member='serviceAccount:service-{project_number}@gcp-sa-iap.iam.gserviceaccount.com'  \
    --role='roles/run.invoker' \
    --region={region}

### Enable IAP on the ADK Agent Cloud Run Service

In [ ]:
!gcloud beta run services update finance-agent \
    --quiet \
    --no-user-output-enabled \
    --region={region} \
    --iap

## Test the Agent

### Trying Accessing the Web UI

Run the cell below to get your web UI URL, and try to access it. Login with your student account when prompted to login. You will get an error saying "You don't have access." This is expected. We'll add permissions in the next cell.

In [ ]:
# Retrieve the Web UI URL
agent_url = !gcloud run services describe finance-agent --region {region} --format 'value(status.url)'
agent_url = agent_url[0]
print(agent_url)

### Add IAP Permissions

Next, we'll add permissions for your student account so that you can securely access the Web UI.

In [ ]:
# Replace the variable value for student_email below with your student email address.
student_email = "admin@paulramsey.altostrat.com"

In [ ]:
# Add IAP access to the ADK Cloud Run service for your student account
!gcloud beta iap web add-iam-policy-binding \
    --member=user:{student_email} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=finance-agent

# Add IAP access to the Toolbox Cloud Run service for your student account
!gcloud beta iap web add-iam-policy-binding \
    --member=user:{student_email} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=toolbox

In [ ]:
# Get the service account running the finance-agent Cloud Run service
adk_service_account = !gcloud run services describe finance-agent \
    --region={region} \
    --format="value(spec.template.spec.serviceAccountName)"
adk_service_account = adk_service_account[0]
print(f"ADK Service Account: {adk_service_account}")

# Get the service account running the toolbox Cloud Run service
toolbox_service_account = !gcloud run services describe toolbox \
    --region={region} \
    --format="value(spec.template.spec.serviceAccountName)"
toolbox_service_account = f"{toolbox_service_account[0]}@{project_id}.iam.gserviceaccount.com"
print(f"Toolbox Service Account: {toolbox_service_account}")

# Dev notes:
# Added student email access to Web UI. Could access the UI, but got a 401 error when ADK tried to call a tool.
# Enabled IAP on toolbox service and granted student email access to toolbox as well. Still got a 401 on tool calls. 
# Disabled IAP on toolbox service. Still got a 401 on tool calls. 
# Configured the Oauth IAP Screen. Works now. 

In [ ]:
# Give ADK service account IAP access to the toolbox service account
# This cell may not be necessary
!gcloud beta iap web add-iam-policy-binding \
    --member=serviceAccount:{adk_service_account} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=toolbox

# Give Toolbox service acconut IAP access to the ADK service account
!gcloud beta iap web add-iam-policy-binding \
    --member=serviceAccount:{toolbox_service_account} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=finance-agent


### Navigate the UI

Now wait for a little over a minute and try access the UI again. This time you should see an interface like the screenshot below.

![ADK UI](img/adk-ui/adk-ui.png)

Try asking the Agent questions to help you with investigating fraud reports. Here are some examples you might try:

> NOTE: If you see errors like the following, try again. These tend to be transient issues. `{"error": "401, message='Attempt to decode JSON with unexpected mimetype: text/html; charset=utf-8', url='https://toolbox-123456789098.us-central1.run.app/api/tool/get_loan_repayments/invoke'"}`

#### Scenario 1: Investigating a Customer's Loan Repayments
This conversation flow utilizes the Spanner `get_loan_repayments` tool to investigate a customer's financial history.

- Initial User Question: "Can you show me the loan repayment history for customer ID 263?"
- Follow-up Question 1: "Thanks. Can you tell me the total amount of the loans this customer has repaid?"
- Follow-up Question 2: "Which account was used for the most recent repayment, and what was the date of that transaction?"
- Follow-up Question 3: "Based on the data, what is the largest single loan amount this customer has taken out?"

#### Scenario 2: Analyzing Suspicious Account Activity
This exchange uses the Spanner `get_account_inflows_outflows` tool to analyze the movement of money.

- Initial User Question: "I need to analyze the account activity for customer 330. Can you get the total inflows and outflows for their accounts?"
- Follow-up Question 1: "Which of their accounts has the highest ratio of inflows to outflows?"
- Follow-up Question 2: "Is the account with the high inflow/outflow ratio currently blocked? And what type of account is it?"
- Follow-up Question 3: "Can you list all the accounts for this customer and their corresponding money flow ratios?"

#### Scenario 3: Tracing Potential Money Laundering
This conversation uses the Spanner  `get_indirect_transfers` and `get_destination_account_audit_events` tools to trace a series of transactions.

- Initial User Question: "I suspect there might be some indirect money transfers between account 75 and account 199. Can you check if there were any transfers between them with 3 to 6 hops?"
- Follow-up Question 1: "How many hops were there in the transfer? Also, can you tell me if either the source or destination account is currently blocked?"
- Follow-up Question 2: "The destination account seems suspicious. Can you pull the latest audit events for the destination account, especially any related to transfers from suspicious accounts?"
- Follow-up Question 3: "From the audit trail, can you identify the timestamps of the audit events and any details associated with them?"

#### Scenario 4: General Database Queries
This conversation flow demonstrates the use of the flexible `get_finance_database_context` tool for more open-ended questions about the finance database in AlloyDB.

- Initial User Question: "How many users are in the database?"
- Follow-up Question 1: "Can you give me a count of all transactions that have been flagged for fraud?"
- Follow-up Question 2: "What are the different types of credit cards available to customers?"
- Follow-up Question 3: "Can you list the top 5 Merchant Category Codes (MCC) by transaction volume?"
